In [37]:
clear()

In [38]:
import chess.pgn

In [ ]:
# YOU HAVE TO DOWNLOAD THE DATASET YOURSELF I CAN'T PUSH IT TO GITHUB ITS TOOOO LARGE

In [12]:
with open('lichess_db_standard_rated_2018-04.pgn') as pgn:
        game = chess.pgn.read_game(pgn)
print(int(game.headers['WhiteElo']) >= 2000)
print(int(game.headers['BlackElo']) >= 2000)

False
False


In [ ]:
with open('lichess_db_standard_rated_2018-04.pgn') as pgn:
    game = chess.pgn.read_game(pgn)
    while game is not None:
        board = game.board()
        for move in game.mainline_moves():
            board.push(move)
        game = chess.pgn.read_game(pgn)

In [ ]:
with open('lichess_db_standard_rated_2018-04.pgn') as pgn:
    game = chess.pgn.read_game(pgn)
#     print(game)
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
        print(str(board) + '\n\n')

In [39]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [40]:
# Define the neural network model
def create_model():
    # Input layer for 12 8x8 boards
#     input_boards = layers.Input(shape=(8, 8, 12))
    input_shape = (8, 8, 14)
    num_units = 64

    # Create the model
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Flatten(),  # Flatten the input into a 1D array
        tf.keras.layers.Conv2D(3, 3, activation = 'relu'),
        tf.keras.layers.Conv2D(3, 3, activation = 'relu'),
        tf.keras.layers.Dense(2048, activation='relu'),  # 1st hidden layer
        tf.keras.layers.Dense(2048, activation='relu'),  # 2nd hidden layer
        tf.keras.layers.Dense(2048, activation='relu'),  # 3rd hidden layer
        tf.keras.layers.Dense(num_units, activation='relu'),  # Final layer before dot product
    ])
    return model

In [41]:
def convert_board_to_bitboards(board):
    bitboards = np.zeros((14, 8, 8), dtype=np.uint8)  # 14 channels for pieces + 2 channels for attacked squares

    for piece_type in range(1, 7):
        for color in [True, False]:
            pieces = board.pieces(piece_type, color)
            for square in pieces:
                index = (piece_type - 1) * 2 + int(not color)
                row, col = divmod(square, 8)
                bitboards[index][row][col] = 1

    # Calculate attacked squares by white and black pieces
    
    white_attacks = [1 if board.attackers(chess.WHITE, square) else 0 for square in chess.SQUARES]
    black_attacks = [1 if board.attackers(chess.BLACK, square) else 0 for square in chess.SQUARES]
    # Add attacked squares into the bitboards
    bitboards[12] = np.reshape(white_attacks, (8, 8))
    bitboards[13] = np.reshape(black_attacks, (8, 8))

    return bitboards.flatten()

In [42]:
def encode_move(move):
    if move is None:
        return np.zeros((8, 8), dtype=np.uint8).flatten()
    # Extract source and destination squares from the move
    source_square = move.from_square
    dest_square = move.to_square
    
    # Create a bitboard representing the move
    move_bitboard = np.zeros((8, 8), dtype=np.uint8)
    move_bitboard[source_square // 8][source_square % 8] = 1
    move_bitboard[dest_square // 8][dest_square % 8] = 1
    
    return move_bitboard.flatten()  # Flatten the bitboard to match the required shape

In [ ]:
with open('lichess_db_standard_rated_2018-04.pgn') as pgn:
    games = []
    game = chess.pgn.read_game(pgn)
    i = 0
    while game is not None and i<50000:
        if int(game.headers['WhiteElo']) >= 2000 or int(game.headers['BlackElo']) >= 2000:
            games.append(game)
        game = chess.pgn.read_game(pgn)
        i+=1

In [ ]:
training_data = []
for game in games:
    board = game.board()
    game_moves = list(game.mainline_moves())
    for i in range(len(game_moves)):
        bits = convert_board_to_bitboards(board)
        if i+1 < len(game_moves):
            training_data.append((bits, encode_move(game_moves[i+1])))
        else:
            training_data.append((bits, encode_move(None)))
        board.push(game_moves[i])

In [ ]:
len(training_data[1][1])

In [ ]:
training_data[1]

In [ ]:
# Convert the training data into NumPy arrays
X_train = np.array([data[0] for data in training_data])
y_train = np.array([data[1] for data in training_data])

# Split the data into training and validation sets
validation_split = 0.2  # Adjust this ratio as needed
split_index = int(len(X_train) * (1 - validation_split))

X_train, X_val = X_train[:split_index], X_train[split_index:]
y_train, y_val = y_train[:split_index], y_train[split_index:]

# Shuffle the training data
shuffle_indices = np.random.permutation(len(X_train))
X_train, y_train = X_train[shuffle_indices], y_train[shuffle_indices]

# Create the model
model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])  # Adjust loss and metrics as needed

# Train the model
batch_size = 32  # Adjust batch size as needed
epochs = 10  # Adjust the number of epochs as needed


new_shape = (-1, 8, 8, 14)  # The -1 in the shape means "infer this dimension"

X_train_reshaped = X_train.reshape(new_shape)
X_val_reshaped = X_val.reshape(new_shape)
new_target_shape = (-1, 64)  # Define the new shape for the target output
y_train_reshaped = y_train.reshape(new_target_shape)
y_val_reshaped = y_val.reshape(new_target_shape)

model.fit(X_train_reshaped, y_train_reshaped, batch_size=batch_size, epochs=epochs, validation_data=(X_val_reshaped, y_val_reshaped))


In [94]:
history = model.fit(X_train_reshaped, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val_reshaped, y_val))
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Epoch 1/10
2185/2185 [==============================] - 194s 89ms/step - loss: 0.0308 - accuracy: 0.0274 - val_loss: 0.0308 - val_accuracy: 0.0275
Epoch 2/10
 928/2185 [===========>..................] - ETA: 1:51 - loss: 0.0308 - accuracy: 0.0265

KeyboardInterrupt: 